In [1]:
import pandas as pd  
import re
import os
import json

In [2]:
df = pd.read_csv('2.0_complete_olympics_athlete.csv')
print(type(df['competition']))

<class 'pandas.core.series.Series'>


In [12]:
competition_info = []
for index, row in df.iterrows():
    competitions_str = row['competition']
    
    # Check if the cell contains the literal string "[]"
    if competitions_str == "[]":
        competition_info.append(None)  # Append None for empty list string
    else:
        competitions = json.loads(competitions_str)  # Parse the 'competition' field
        # Check if 'competitions' is a non-empty list and contains 'competition' key
        if competitions and isinstance(competitions, list) and len(competitions) > 0:
            if 'competition' in competitions[0]:  # Ensure 'competition' key exists in the first entry
                competition_info.append(competitions[0]['competition'])
            else:
                competition_info.append(None)  # Append None if 'competition' key is missing
        else:
            competition_info.append(None)  # Append None for empty lists or invalid data

# Create a new DataFrame with the extracted competition_info
competition_df = pd.DataFrame({'competition_info': competition_info})

# Concatenate the new DataFrame with the existing one
df_combined = pd.concat([df, competition_df], axis=1)

# Export the combined DataFrame to a CSV file
df_combined.to_csv('2.1_complete_olympics_athlete_extracted.csv', index=False)


In [13]:
# Define a function to split competition info
def split_competitions(text):
    if isinstance(text, str):  # Ensure text is a string
        # Updated pattern to allow single or double digit hours
        pattern = re.compile(r'(\d+ (?:juil\.|août)\d{1,2}:\d{2}.*?Terminé\(e\))', re.DOTALL)
        matches = pattern.findall(text)
        return matches
    return []  # Return an empty list if text is not a string


In [14]:
# Apply the function to each row and convert the results into a DataFrame
competition_df = df_combined['competition_info'].apply(split_competitions)
competition_df.head()
competition_df.to_csv('split_sample.csv')

In [15]:
# Determine the maximum number of competitions in any row to define the number of columns
max_competitions = competition_df.apply(len).max()

# Create a new DataFrame where each competition is a separate column
competition_df_expanded = pd.DataFrame(competition_df.tolist(), columns=[f'Competition_{i+1}' for i in range(max_competitions)])

# Concatenate the original DataFrame with the new columns
df_combined_split = pd.concat([df, competition_df_expanded], axis=1)

df_combined_split.to_csv('2.2_complete_olympics_athlete_extracted_split.csv')

In [16]:
# Define a function to extract the date from competition text
def extract_date(text):
    if isinstance(text, str):
        # Regex pattern to extract the date part (e.g., "9 juil." or "9 août")
        pattern = re.compile(r'(\d+ (?:juil\.|août))')
        match = pattern.search(text)
        if match:
            return match.group(1)
    return None

# Assuming df_combined_split is your DataFrame with competition columns
# Apply the function to each competition column
competition_columns = [col for col in df_combined_split.columns if col.startswith('Competition_')]

for col in competition_columns:
    df_combined_split[f'{col}_Date'] = df_combined_split[col].apply(extract_date)

# Save the updated DataFrame to a CSV file
df_combined_split.to_csv('2.3_complete_olympics_athlete_extracted_split_date.csv', index=False)


In [17]:
df_combined_split.head()

,web-scraper-order,web-scraper-start-url,nom,lien_vers_biographie,lien_vers_biographie-href,discipline,naissance,age,fonction,taille,...,Competition_7_Date,Competition_8_Date,Competition_9_Date,Competition_10_Date,Competition_11_Date,Competition_12_Date,Competition_13_Date,Competition_14_Date,Competition_15_Date,Competition_16_Date
0,1727352275-1,https://olympics.com/fr/paris-2024/athletes,671,671 671,https://olympics.com/fr/paris-2024/athlete/-67...,Breaking,19 oct. 2005,Âge: 18,Fonction: Athlète,NaN,...,None,None,None,None,None,None,None,None,None,None
1,1727352277-2,https://olympics.com/fr/paris-2024/athletes,ZWOLINSKA Klaudia,ZWOLINSKA Klaudia ZWOLINSKA K,https://olympics.com/fr/paris-2024/athlete/kla...,Canoë-kayak slalom,18 déc. 1998,Âge: 25,Fonction: Athlète,NaN,...,31 juil.,2 août,3 août,3 août,4 août,None,None,None,None,None
2,1727352279-3,https://olympics.com/fr/paris-2024/athletes,ZWICKER Martin,ZWICKER Martin ZWICKER M,https://olympics.com/fr/paris-2024/athlete/mar...,Hockey,27 févr. 1987,Âge: 37,Fonction: Athlète,NaN,...,6 août,8 août,None,None,None,None,None,None,None,None
3,1727352281-4,https://olympics.com/fr/paris-2024/athletes,ZWETSLOOT Roos,ZWETSLOOT Roos ZWETSLOOT R,https://olympics.com/fr/paris-2024/athlete/roo...,Skateboard,27 juil. 2000,Âge: 24,Fonction: Athlète,NaN,...,None,None,None,None,None,None,None,None,None,None
4,1727352283-5,https://olympics.com/fr/paris-2024/athletes,ZVEREV Alexander,ZVEREV Alexander ZVEREV A,https://olympics.com/fr/paris-2024/athlete/ale...,Tennis,20 avr. 1997,Âge: 27,Fonction: Athlète,Taille mètres/pieds: 1.98/6'6'',...,None,None,None,None,None,None,None,None,None,None


In [18]:
# Define a function to clean the DataFrame using regex
# Define a function to clean the DataFrame using regex

# Define a function to clean the DataFrame using regex
def clean_data_with_regex(df):
    # Prepare to store cleaned data
    cleaned_data = {
        'cleaned_age': [],
        'cleaned_fonction': [],
        'cleaned_taille': [],
        'cleaned_sexe': []
    }
    
    for index, row in df.iterrows():
        # Transform age into string
        if isinstance(row['age'], (int, float)):
            age_str = str(row['age'])
        else:
            age_str = row['age']
        # Clean the 'age' part using regex to extract the digits
        age_match = re.search(r'\d+', age_str)  # Extract digits
        cleaned_data['cleaned_age'].append(age_match.group(0) if age_match else None)
        
        # Transform 'fonction' to string
        if isinstance(row['fonction'], str):  # Vérifier si c'est une chaîne
            fonction_match = re.search(r'Fonction:\s*(.*)', row['fonction'])  # Capturer après "Fonction:"
            cleaned_data['cleaned_fonction'].append(fonction_match.group(1).strip() if fonction_match else None)
        else:
            cleaned_data['cleaned_fonction'].append(None) 
        # Clean the 'fonction' part using regex to extract the job title
        #fonction_match = re.search(r'Fonction:\s*(.*)', row['fonction'])  # Capture everything after "Fonction:"
        #cleaned_data['cleaned_fonction'].append(fonction_match.group(1).strip() if fonction_match else None)
        
        # Clean the 'taille' part using regex to extract the meter value only
        taille_value = row['taille'] if isinstance(row['taille'], str) else None  # Ensure it's a string
        taille_match = re.search(r'Taille mètres/pieds:\s*(\d+\.\d+|\d+)', taille_value) if taille_value else None
        
        cleaned_data['cleaned_taille'].append(taille_match.group(1) if taille_match else None)
       
        # Nettoyage de la colonne 'sexe'
        if isinstance(row['sexe'], str):  # Vérifier si c'est une chaîne
            sexe_match = re.search(r'Sexe:\s*(.*)', row['sexe'])  # Capturer après "Sexe:"
            cleaned_data['cleaned_sexe'].append(sexe_match.group(1).strip() if sexe_match else None)
        else:
            cleaned_data['cleaned_sexe'].append(None)
        
        # Clean the 'sexe' part using regex to extract gender
        #sexe_match = re.search(r'Sexe:\s*(.*)', row['sexe'])  # Capture everything after "Sexe:"
        #cleaned_data['cleaned_sexe'].append(sexe_match.group(1).strip() if sexe_match else None)

    # Add cleaned data as new columns to the original DataFrame
    for key in cleaned_data:
        df[key] = cleaned_data[key]
    
    return df

# Clean the data
df_combined_split_clean = clean_data_with_regex(df_combined_split)

# Optionally, drop the original dirty columns if you only need the cleaned columns
df_combined_split_clean.drop(['age', 'fonction', 'taille', 'sexe'], axis=1, inplace=True)
# Display the cleaned DataFrame
df_combined_split_clean.head(20)

df_combined_split_clean.to_csv('2.4_complete_olympics_athlete_extracted_split_date_clean.csv', index=False)


## Extract olympic sites using LLMs